In [ ]:
import os, glob
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt

In [ ]:
npy_paths = glob.glob("data/training_processed/patient*/*.npy")
len(npy_paths)

In [ ]:
MM = []
mm = []
for p in tqdm(npy_paths):
    npy = np.load(p)
    # npy = np.clip(npy, -200, 2000)
    M = npy.max()
    m = npy.min()
    MM.append(M)
    mm.append(m)

In [ ]:
H = plt.hist(MM)

In [ ]:
h = plt.hist(mm)

In [ ]:
from train_BVF_pl import Model_factory
from dataset import pyramid_dataset
import argparse
from torch.utils.data import DataLoader
from multiprocessing import cpu_count
import torch

In [ ]:
def parse():
    # Argument Parser
    parser = argparse.ArgumentParser()
    # dataset information
    parser.add_argument('--train_folder', default='data/training_processed')
    parser.add_argument('--test_folder', default='data/testing_4')
    # training hyper-parameter
    parser.add_argument('--model_name', required=True)
    parser.add_argument('-b', '--batch_size', default=8, type=int)
    parser.add_argument('-l', '--lr', default=1e-3,  type=float)
    parser.add_argument('-e', '--epochs', default=2000, type=int)
    # model hyper-parameter
    parser.add_argument('--resume', action="store_true")
    parser.add_argument('--train', action="store_true")
    parser.add_argument('--test', action="store_true")
    
    return parser.parse_args(args=['--model_name', '0617', '--train'])
args = parse()

In [ ]:
Model = Model_factory.load_from_checkpoint('lightning_logs/0617/checkpoints/epoch=1999-step=14000.ckpt', args=args)

In [ ]:
valDataset = pyramid_dataset.dataset(args.test_folder, 0.5, train=False)
valDataloader = DataLoader(dataset=valDataset,
                           batch_size=1,
                           shuffle=True, 
                           num_workers=cpu_count())

In [ ]:
for batch in valDataloader:
    I0, I1, It, t = batch
    I0 = I0.to('cuda')
    I1 = I1.to('cuda')
    It = It.to('cuda')
    t = t.to('cuda')
    break

In [ ]:
flow, flow_1_2, flow_1_4 = Model.model(I0, I1, t)
flowt0, flowt1, It_0, It_1 = Model.warp(flow, I0, I1)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(10, 5))
imgs = I0[0, 0].cpu().detach().numpy()
for i in range(1, 11):
    plt.subplot(2, 5, i)
    plt.axis('off')
    plt.imshow(imgs[i], cmap='gray', vmin=-1, vmax=1)

In [ ]:
plt.figure(figsize=(10, 5))
imgs = It_0[0, 0].cpu().detach().numpy()
for i in range(1, 11):
    plt.subplot(2, 5, i)
    plt.axis('off')
    plt.imshow(imgs[i], cmap='gray', vmin=-1, vmax=1)

In [ ]:
plt.figure(figsize=(10, 5))
imgs = It[0, 0].cpu().detach().numpy()
for i in range(1, 11):
    plt.subplot(2, 5, i)
    plt.axis('off')
    plt.imshow(imgs[i], cmap='gray', vmin=-1, vmax=1)

In [ ]:
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim
from skimage.metrics import mean_squared_error as compare_mse
import torch
from torchmetrics.functional import structural_similarity_index_measure
from torchmetrics.functional import mean_squared_error

In [ ]:
newI0 = Model.eliminate_border(It_0, I0)
pred = Unormalize(newI0)[:, :, 1:-1, :, :]
gt = Unormalize(It)[:, :, 1:-1, :, :]
structural_similarity_index_measure(pred, gt, data_range=1), mean_squared_error(pred, gt)

In [ ]:
def Unormalize(x):
    x = x.clip(-1, 1)
    x = (x+1) / 2
    return x

In [ ]:
newI0 = Model.eliminate_border(It_0, I0)

In [ ]:
pred = Unormalize(newI0)[0, 0, 1:-1].cpu().detach().numpy()
gt = Unormalize(It)[0, 0, 1:-1].cpu().detach().numpy()
compare_psnr(pred, gt, data_range=1)

In [ ]:
compare_ssim(pred, gt, data_range=1)

In [ ]:
pred.min(), pred.max(), gt.min(), gt.max()

In [ ]:
import numpy as np

In [ ]:
one = "data/testing_4/patient102/07.npy"
two = "data/testing_4/patient102/17.npy"

In [ ]:
o = np.load(one)
t = np.load(two)

In [ ]:
o_mean_normalize = (o - o.mean()) / (o.max() - o.min())
t_mean_normalize = (t - t.mean()) / (t.max() - t.min())
o_mean_normalize.max(), o_mean_normalize.min()

In [ ]:
o_min_normalize = (o - o.min()) / (o.max() - o.min())
t_min_normalize = (t - t.min()) / (t.max() - t.min())
o_min_normalize.max(), o_min_normalize.min()

In [ ]:
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

In [ ]:
compare_psnr(o_mean_normalize, t_mean_normalize)

In [ ]:
compare_psnr(o_min_normalize, t_min_normalize)

In [ ]:
def unormalize(x):
    return (x - x.mean()) / (x.max() - x.min())

In [ ]:
compare_psnr(unormalize(o_min_normalize), unormalize(t_min_normalize))

In [7]:
import glob, os
patient_paths = glob.glob('data/training/patient*') + glob.glob('data/testing/patient*')
patient_paths[:5]

['data/training/patient039',
 'data/training/patient036',
 'data/training/patient092',
 'data/training/patient025',
 'data/training/patient069']

In [8]:
import random
random.shuffle(patient_paths)
patient_paths[:5]

['data/training/patient074',
 'data/training/patient043',
 'data/training/patient020',
 'data/training/patient040',
 'data/testing/patient113']

In [10]:
import shutil
folder = "data/training-2/"
if not os.path.isdir(folder):
    os.makedirs(folder)
for patient_path in patient_paths[:100]:
    print(patient_path)
    patient_name = patient_path.split('/')[-1]
    shutil.copytree(patient_path, "data/training-2/{}".format(patient_name))

folder = "data/testing-2/"
if not os.path.isdir(folder):
    os.makedirs(folder)
for patient_path in patient_paths[100:]:
    print(patient_path)
    patient_name = patient_path.split('/')[-1]
    shutil.copytree(patient_path, "data/testing-2/{}".format(patient_name))

data/training/patient074
data/training/patient043
data/training/patient020
data/training/patient040
data/testing/patient113
data/training/patient041
data/testing/patient107
data/training/patient077
data/training/patient066
data/training/patient018
data/testing/patient143
data/training/patient036
data/training/patient005
data/training/patient045
data/testing/patient142
data/training/patient097
data/testing/patient102
data/training/patient034
data/testing/patient135
data/testing/patient101
data/testing/patient125
data/testing/patient127
data/training/patient026
data/testing/patient112
data/training/patient085
data/training/patient062
data/testing/patient129
data/testing/patient106
data/training/patient090
data/training/patient010
data/training/patient067
data/testing/patient148
data/testing/patient126
data/training/patient009
data/testing/patient117
data/testing/patient133
data/testing/patient115
data/training/patient057
data/training/patient071
data/training/patient050
data/training/pat